In [2]:
import numpy as np
import cv2 as cv
import matplotlib.pyplot as plt

In [20]:
# Kernels & Convolution

def mean(size):
    size_x, size_y = size[0], size[1]
    return np.full((size_x, size_y), 1 / (size_x * size_y))

def median(size):
    res = np.zeros((size, size))
    res[int(size / 2)][int(size / 2)] = 1
    return res

def gaussian(size, sigma):
    size_x, size_y = size[0], size[1]
    X, Y = np.meshgrid(np.arange(int(-size_x / 2),  int(size_x / 2) + 1,  1), np.arange(int(-size_y / 2),  int(size_y / 2) + 1,  1))
    res = (1 / (2 * np.pi * np.power(sigma, 2))) * np.exp(-(np.power(X, 2) + np.power(Y, 2)) / (2 * np.power(sigma, 2)))
    return res / sum(sum(res))

def dxy(vertical):
    if vertical:
        return np.array([[-1, 1]], dtype=np.int32)
    return np.array([[-1], [1]], dtype=np.int32)

def prewitt(vertical):
    if vertical:
        return np.array([[-1, 0, 1], [-1, 0, 1], [-1, 0, 1]], dtype=np.int32)
    return np.array([[1, 1, 1], [0, 0, 0], [-1, -1, -1]], dtype=np.int32) 

def sobel(vertical):
    if vertical:
        return np.array([[-1, 0, 1], [-2, 0, 2], [-1, 0, 1]], dtype=np.int32)
    return np.array([[1, 2, 1], [0, 0, 0], [-1, -2, -1]], dtype=np.int32)


def roberts(vertical):
    if vertical:
        return np.array([[0, 1], [-1, 0]], dtype=np.int32)
    return np.array([[1, 0], [0, -1]], dtype=np.int32) 

def convolution2d(image, kernel, padding, stride, isMedian):
    image = np.pad(image, [(padding, padding), (padding, padding)], mode='constant', constant_values = 0)

    kernel_height, kernel_width = kernel.shape
    padded_height, padded_width = image.shape

    output_height = (padded_height - kernel_height) // stride + 1
    output_width = (padded_width - kernel_width) // stride + 1

    new_image = np.zeros((output_height, output_width)).astype(np.float32)

    kernel = np.flip(np.flip(kernel, 0), 1)
    
    if isMedian:
        for y in range(output_height):
            for x in range(output_width):
                new_image[y][x] = np.sum(np.median(image[y * stride : y * stride + kernel_height, x * stride : x * stride + kernel_width]) * kernel).astype(np.float32)
    else:
        for y in range(output_height):
            for x in range(output_width):
                new_image[y][x] = np.sum(image[y * stride : y * stride + kernel_height, x * stride : x * stride + kernel_width] * kernel).astype(np.float32)
                
    return new_image

def magnitude(image, kernel_x, kernel_y, padding, stride):
    image = np.pad(image, [(padding, padding), (padding, padding)], mode='constant', constant_values = 0)

    kernel_height, kernel_width = kernel_x.shape
    padded_height, padded_width = image.shape

    output_height = (padded_height - kernel_height) // stride + 1
    output_width = (padded_width - kernel_width) // stride + 1

    new_image = np.zeros((output_height, output_width)).astype(np.float32)
    
    for y in range(output_height):
        for x in range(output_width):
            dx = np.sum(image[y * stride : y * stride + kernel_height, x * stride : x * stride + kernel_width] * kernel_x).astype(np.float32)
            dy = np.sum(image[y * stride : y * stride + kernel_height, x * stride : x * stride + kernel_width] * kernel_y).astype(np.float32)
            new_image[y][x] = np.sqrt(dx**2 + dy**2)
            
    return new_image


def orientation(image, kernel_x, kernel_y, padding, stride):
    image = np.pad(image, [(padding, padding), (padding, padding)], mode='constant', constant_values = 0)

    kernel_height, kernel_width = kernel_x.shape
    padded_height, padded_width = image.shape

    output_height = (padded_height - kernel_height) // stride + 1
    output_width = (padded_width - kernel_width) // stride + 1

    new_image = np.zeros((output_height, output_width)).astype(np.float32)
    
    for y in range(output_height):
        for x in range(output_width):
            dx = np.sum(image[y * stride : y * stride + kernel_height, x * stride : x * stride + kernel_width] * kernel_x).astype(np.float32)
            dy = np.sum(image[y * stride : y * stride + kernel_height, x * stride : x * stride + kernel_width] * kernel_y).astype(np.float32)
            new_image[y][x] = np.arctan2(-dy, dx)
            
    return new_image

def seam(image, kernel_x, kernel_y, padding, stride):
    image = np.pad(image, [(padding, padding), (padding, padding)], mode='constant', constant_values = 0)

    kernel_height, kernel_width = kernel_x.shape
    padded_height, padded_width = image.shape

    output_height = (padded_height - kernel_height) // stride + 1
    output_width = (padded_width - kernel_width) // stride + 1

    new_image = np.zeros((output_height, output_width)).astype(np.float32)
    
    for y in range(output_height):
        for x in range(output_width):
            dx = np.sum(image[y * stride : y * stride + kernel_height, x * stride : x * stride + kernel_width] * kernel_x).astype(np.float32)
            dy = np.sum(image[y * stride : y * stride + kernel_height, x * stride : x * stride + kernel_width] * kernel_y).astype(np.float32)
            new_image[y][x] = np.abs(dx) + np.abs(dy)
            
    return new_image

In [11]:
img1 = cv.imread('./images/VanGogh.jpg')
img1 = cv.cvtColor(img1, cv.COLOR_BGR2GRAY)

img2 = cv.imread('./images/VanGogh.jpg')
img2 = cv.cvtColor(img2, cv.COLOR_BGR2GRAY)

for i in range(3):
    dsp1 = img1[::2, ::2]

    x, y = 3, 3
    blur = convolution2d(img2, gaussian((3, 3), 3), 1, 1, False)
    # blur = cv.GaussianBlur(img2, (x, y), 3, borderType=cv.BORDER_CONSTANT)
    dsp2 = blur[::2, ::2]
    x *= 2 + 1
    y *= 2 + 1
    
    img1 = dsp1
    img2 = dsp2
    
    cv.imwrite(f'VanGogh_DSPWithoutBlur{i + 1}.jpg', dsp1)
    cv.imwrite(f'VanGogh_DSPWithBlur{i + 1}.jpg', dsp2)

In [12]:
# 3 blurring methods
img = cv.imread('./images/VanGogh.jpg')
img = cv.cvtColor(img, cv.COLOR_BGR2GRAY)

size = (3, 3)
sigma = 3

convo_mean = convolution2d(img, mean(size), 1, 1, False)
# convo_mean = cv.blur(img, (3, 3), borderType=cv.BORDER_CONSTANT)

convo_median = convolution2d(img, median(size[0]), 0, 1, True)
# convo_median = cv.medianBlur(img, 3)

convo_gaussian = convolution2d(img, gaussian(size, sigma), 1, 1, False)
# convo_gaussian = cv.GaussianBlur(img, (3, 3), 3, 3, borderType=cv.BORDER_CONSTANT)

cv.imwrite('VanGogh_MeanBlur.jpg', convo_mean)
cv.imwrite('VanGogh_MedianBlur.jpg', convo_median)
cv.imwrite('VanGogh_GaussianBlur.jpg', convo_gaussian)

True

In [9]:
# Derivative, Orientation, and Seam Carving

img = cv.imread('.W/images/Den.jpg')
img = cv.cvtColor(img, cv.COLOR_BGR2GRAY)

convo_dx = convolution2d(img, dxy(True), 1 , 1, False)
convo_dy = convolution2d(img, dxy(False), 1 , 1, False)
convo_df = magnitude(img, dxy(True), dxy(False), 1, 1)
# orientation_df = orientation(img, dxy(True), dxy(False), 1, 1)
seam_df = seam(img, dxy(True), dxy(False), 1, 1)

convo_prewitt_dx = convolution2d(img, prewitt(True), 1 , 1, False)
convo_prewitt_dy = convolution2d(img, prewitt(False), 1 , 1, False)
convo_prewitt = magnitude(img, prewitt(True), prewitt(False), 1, 1)
orientation_prewitt = orientation(img, prewitt(True), prewitt(False), 1, 1)
seam_prewitt = seam(img, prewitt(True), prewitt(False), 1, 1)

convo_sobel_dx = convolution2d(img, sobel(True), 1 , 1, False)
convo_sobel_dy = convolution2d(img, sobel(False), 1 , 1, False)
convo_sobel = magnitude(img, sobel(True), sobel(False), 1, 1)
# orientation_sobel = orientation(img, sobel(True), sobel(False), 1, 1)
seam_sobel = seam(img, sobel(True), sobel(False), 1, 1)

convo_roberts_dx = convolution2d(img, roberts(True), 1 , 1, False)
convo_roberts_dy = convolution2d(img, roberts(False), 1 , 1, False)
convo_roberts = magnitude(img, roberts(True), roberts(False), 1, 1)
# orientation_roberts = orientation(img, roberts(True), roberts(False), 1, 1)
seam_roberts = seam(img, roberts(True), roberts(False), 1, 1)

cv.imwrite('VanGogh_dx.jpg', convo_dx)
cv.imwrite('VanGogh_dy.jpg', convo_dy)
cv.imwrite('VanGogh_df.jpg', convo_df)
# cv.imwrite('VanGogh_Orientation_df.jpg', orientation_df)
cv.imwrite('VanGogh_Seam_df.jpg', seam_df)

cv.imwrite('VanGogh_Prewitt_dx.jpg', convo_prewitt_dx)
cv.imwrite('VanGogh_Prewitt_dy.jpg', convo_prewitt_dy)
cv.imwrite('VanGogh_Prewitt.jpg', convo_prewitt)
cv.imwrite('VanGogh_Orientation_Prewitt.jpg', orientation_prewitt)
cv.imwrite('VanGogh_Seam_Prewitt.jpg', seam_prewitt)

cv.imwrite('VanGogh_Sobel_dx.jpg', convo_sobel_dx)
cv.imwrite('VanGogh_Sobel_dy.jpg', convo_sobel_dy)
cv.imwrite('VanGogh_Sobel.jpg', convo_sobel)
# cv.imwrite('VanGogh_Orientation_Sobel.jpg', orientation_sobel)
cv.imwrite('VanGogh_Seam_Sobel.jpg', seam_sobel)

cv.imwrite('VanGogh_Roberts_dx.jpg', convo_roberts_dx)
cv.imwrite('VanGogh_Roberts_dy.jpg', convo_roberts_dy)
cv.imwrite('VanGogh_Roberts.jpg', convo_roberts)
# cv.imwrite('VanGogh_Orientation_Roberts.jpg', orientation_roberts)
cv.imwrite('VanGogh_Seam_Roberts.jpg', seam_roberts)

error: OpenCV(4.7.0) D:\a\opencv-python\opencv-python\opencv\modules\imgproc\src\color.cpp:182: error: (-215:Assertion failed) !_src.empty() in function 'cv::cvtColor'


In [8]:
# Derivative with smoothing
img = cv.imread('./images/VanGogh.jpg')
img = cv.cvtColor(img, cv.COLOR_BGR2GRAY)

size = (3, 3)
sigma = 3

dx_gaussian = convolution2d(gaussian(size, sigma), dxy(True), 1 , 1, False)
dy_gaussian = convolution2d(gaussian(size, sigma), dxy(False), 1 , 1, False)
convo_dx = convolution2d(img, dx_gaussian, 1, 1, False)
convo_dy = convolution2d(img, dy_gaussian, 1, 1, False)
convo_df = magnitude(img, dx_gaussian, dy_gaussian.T, 1, 1)

prewitt_dx_gaussian = convolution2d(gaussian(size, sigma), dxy(True), 1 , 1, False)
prewitt_dy_gaussian = convolution2d(gaussian(size, sigma), dxy(False), 1 , 1, False)
convo_prewitt_dx = convolution2d(img, prewitt_dx_gaussian, 1, 1, False)
convo_prewitt_dy = convolution2d(img, prewitt_dy_gaussian, 1, 1, False)
convo_prewitt = magnitude(img, prewitt_dx_gaussian, prewitt_dy_gaussian.T, 1, 1)

sobel_dx_gaussian = convolution2d(gaussian(size, sigma), dxy(True), 1 , 1, False)
sobel_dy_gaussian = convolution2d(gaussian(size, sigma), dxy(False), 1 , 1, False)
convo_sobel_dx = convolution2d(img, sobel_dx_gaussian, 1, 1, False)
convo_sobel_dy = convolution2d(img, sobel_dy_gaussian, 1, 1, False)
convo_sobel = magnitude(img, sobel_dx_gaussian, sobel_dy_gaussian.T, 1, 1)

roberts_dx_gaussian = convolution2d(gaussian(size, sigma), dxy(True), 1 , 1, False)
roberts_dy_gaussian = convolution2d(gaussian(size, sigma), dxy(False), 1 , 1, False)
convo_roberts_dx = convolution2d(img, roberts_dx_gaussian, 1, 1, False)
convo_roberts_dy = convolution2d(img, roberts_dy_gaussian, 1, 1, False)
convo_roberts = magnitude(img, roberts_dx_gaussian, roberts_dy_gaussian.T, 1, 1)

cv.imwrite('VanGogh_dx_Gaussian.jpg', convo_dx)
cv.imwrite('VanGogh_dy_Gaussian.jpg', convo_dy)
cv.imwrite('VanGogh_df_Gaussian.jpg', convo_df)

cv.imwrite('VanGogh_Prewitt_dx.jpg', convo_prewitt_dx)
cv.imwrite('VanGogh_Prewitt_dy.jpg', convo_prewitt_dy)
cv.imwrite('VanGogh_Prewitt.jpg', convo_prewitt)

cv.imwrite('VanGogh_Sobel_dx.jpg', convo_sobel_dx)
cv.imwrite('VanGogh_Sobel_dy.jpg', convo_sobel_dy)
cv.imwrite('VanGogh_Sobel.jpg', convo_sobel)

cv.imwrite('VanGogh_Roberts_dx.jpg', convo_roberts_dx)
cv.imwrite('VanGogh_Roberts_dy.jpg', convo_roberts_dy)
cv.imwrite('VanGogh_Roberts.jpg', convo_roberts)

True